In [6]:
import os
import pandas as pd
from skimage import io
from torchmetrics.functional import structural_similarity_index_measure as ssim
import lpips 
import torch
from torchvision import transforms
from skimage import io
import numpy as np
import pandas as pd
import os


In [7]:

# Initialize the LPIPS model for CPU
lpips_model = lpips.LPIPS(net='alex').to('cpu')  # Specify .to('cpu') to ensure it's on CPU

def load_image(image_path, size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((size, size))  # Resize images for consistency
    ])
    image = io.imread(image_path)
    if len(image.shape) == 2:  # Convert grayscale to RGB
        image = np.stack((image,)*3, axis=-1)
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension, ensure it's on CPU

def calculate_ssim(image1, image2):
    return ssim(image1, image2, data_range=1.0).item()

def calculate_lpips(image1, image2, model):
    """Calculate LPIPS similarity between two images, ensuring computation on CPU."""
    distance = model(image1, image2)
    return distance.item()

# Directory paths
style_image_path = 'images/Vassily_Kandinsky_1913_-_Composition_7.jpg'
content_image_path = 'images/YellowLabradorLooking_new.jpg' 
stylized_dir = 'TO_Test'

# Load content and style images
content_image = load_image(content_image_path)
style_image = load_image(style_image_path)

image_filenames = os.listdir(stylized_dir)

# Initialize a dataframe
df = pd.DataFrame(columns=['Image','SSIM_Content_Stylized', 'SSIM_Style_Stylized', 'LPIPS_Content_Stylized', 'LPIPS_Style_Stylized'])

for filename in image_filenames:
    stylized_image_path = os.path.join(stylized_dir, filename)
    
    # Load images
    stylized_image = load_image(stylized_image_path)
    
    # Calculate SSIM
    ssim_content_stylized = calculate_ssim(content_image, stylized_image)
    ssim_style_stylized = calculate_ssim(style_image, stylized_image)
    
    # Calculate LPIPS
    lpips_content_stylized = calculate_lpips(content_image, stylized_image, lpips_model)
    lpips_style_stylized = calculate_lpips(style_image, stylized_image, lpips_model)
    
    # Append new metrics to your DataFrame
    temp_df = pd.DataFrame({
        'Image': [filename],
        'SSIM_Content_Stylized': [ssim_content_stylized],
        'SSIM_Style_Stylized': [ssim_style_stylized],
        'LPIPS_Content_Stylized': [lpips_content_stylized],
        'LPIPS_Style_Stylized': [lpips_style_stylized]
    })
    
    df = pd.concat([df, temp_df], ignore_index=True)

print(df)
df.to_csv('style_transfer_evaluation_results.csv', index=False)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\agish\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\agish\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\agish\AppData\Roaming\Python\Python311\site-packages\lpips\weights\v0.1\alex.pth


C:\Users\agish\AppData\Roaming\Python\Python311\site-packages\torchmetrics\utilities\prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(
C:\Users\agish\AppData\Local\Temp\ipykernel_8592\3274831208.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


                                   Image  SSIM_Content_Stylized  \
0             final_image_high_style.png               0.103495   
1              final_image_low_style.png               0.248896   
2           final_image_medium_style.png               0.127011   
3              final_image_one_style.png               0.104304   
4   optimized_texture_lambda_0.01_19.png               0.072074   
5    optimized_texture_lambda_0.1_19.png               0.186903   
6    optimized_texture_lambda_0.5_19.png               0.283619   
7    optimized_texture_lambda_1.0_19.png               0.270222   
8   optimized_texture_lambda_10.0_19.png               0.262274   
9  optimized_texture_lambda_100.0_19.png               0.394982   

   SSIM_Style_Stylized  LPIPS_Content_Stylized  LPIPS_Style_Stylized  
0             0.021157                0.764977              0.417668  
1             0.020347                0.594752              0.464101  
2             0.023764                0.735808   

In [8]:
df

,Image,SSIM_Content_Stylized,SSIM_Style_Stylized,LPIPS_Content_Stylized,LPIPS_Style_Stylized
0,final_image_high_style.png,0.103495,0.021157,0.764977,0.417668
1,final_image_low_style.png,0.248896,0.020347,0.594752,0.464101
2,final_image_medium_style.png,0.127011,0.023764,0.735808,0.409485
3,final_image_one_style.png,0.104304,0.020578,0.764103,0.410221
4,optimized_texture_lambda_0.01_19.png,0.072074,0.035123,0.610063,0.291381
5,optimized_texture_lambda_0.1_19.png,0.186903,0.036702,0.568409,0.321124
6,optimized_texture_lambda_0.5_19.png,0.283619,0.027888,0.549586,0.353494
7,optimized_texture_lambda_1.0_19.png,0.270222,0.026421,0.530652,0.370212
8,optimized_texture_lambda_10.0_19.png,0.262274,0.023171,0.457469,0.511135
9,optimized_texture_lambda_100.0_19.png,0.394982,0.036040,0.125170,0.560232
